Part A: Data Loading & Basic Operations

Basic Exploration


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

cdf=pd.read_csv('customers.csv')
pdf=pd.read_csv('products.csv')
tdf=pd.read_csv('transactions.csv')
dfs=[cdf,pdf,tdf]
for i, df in enumerate(dfs, 1):
    print(f"\n{'='*20} Dataset {i} {'='*20}")
    print("Shape:", df.shape)
    print("Data Types:\n", df.dtypes)
    print("First 3 Rows:")
    print(df.head(3))
    print("Missing values (per column):")
    print(df.isnull().sum())
num_duplicates = tdf['transaction_id'].duplicated().sum()
print(f"Number of duplicate transaction_id entries: {num_duplicates}")
dates = pd.to_datetime(tdf['timestamp'], errors='coerce')
earliest = df['timestamp'].min()
latest = df['timestamp'].max()

print(f"The date range is :{earliest} to {latest}" )


Datetime Transformation

In [ ]:

df['timestamp'] = pd.to_datetime(tdf['timestamp'], errors='coerce')
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['month'] = df['timestamp'].dt.month
print(df[['timestamp', 'hour', 'day_of_week', 'month']].head())

Part B: Data Cleaning & Transformation

Handle Missing Values

In [ ]:
for col in tdf.select_dtypes(include='number').columns:
    median_val = tdf[col].median()
    tdf[col].fillna(median_val, inplace=True)

tdf['payment_method'].fillna(tdf['payment_method'].mode, inplace=True)
pdf['category'].fillna(pdf['category'].mode, inplace=True)

cdf['age'].fillna(cdf['age'].mean, inplace=True)
cdf['email'].fillna(cdf['email'].mode, inplace=True)

In [ ]:
tdf['revenue'] = tdf['quantity'] * tdf['price']
full_data = tdf.merge(cdf, on='customer_id', how='left')
full_data = full_data.merge(pdf, on='product_id', how='left')
full_data['profit_margin'] = ((full_data['price'] - full_data['cost_price']) / full_data['price']) * 100

full_data.head(5)



Part C: Aggregation & Analysis

C1. Customer Metrics


In [ ]:
def most_frequent_category(series):
    return series.mode().iat[0]
customer_summary = full_data.groupby('customer_id').agg(
    total_revenue=pd.NamedAgg(column='revenue', aggfunc='sum'),
    num_transactions=pd.NamedAgg(column='transaction_id', aggfunc='count'),
    avg_transaction_value=pd.NamedAgg(column='revenue', aggfunc='mean'),
    most_freq_category=pd.NamedAgg(column='category', aggfunc=most_frequent_category)
)

customer_summary = customer_summary.sort_values(by='total_revenue', ascending=False)

print(customer_summary.head())

C2. Time-Based Analysis


In [ ]:
def most_frequent_category(series):
    return series.mode().iat[0]
monthly_summary = full_data.groupby('month').agg(
    total_revenue=pd.NamedAgg(column='revenue', aggfunc='sum'),
    num_unique_customers=pd.NamedAgg(column='customer_id', aggfunc='nunique'),
    avg_order_value=pd.NamedAgg(column='revenue', aggfunc='mean'),
).reset_index()

monthly_summary['mom_growth_pct'] = monthly_summary['total_revenue'].pct_change() * 100

print(customer_summary.head())


Product performance


In [ ]:
product_perf = full_data.groupby('product_id').agg(
    total_revenue=pd.NamedAgg(column='revenue', aggfunc='sum'),
    total_quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
    avg_profit_margin=pd.NamedAgg(column='profit_margin', aggfunc='mean')
).reset_index()

top10_revenue = product_perf.nlargest(10, 'total_revenue')
top10_quantity = product_perf.nlargest(10, 'total_quantity')
top10_profit_margin = product_perf.nlargest(10, 'avg_profit_margin')

print("Top 10 Products by Total Revenue:")
print(top10_revenue)

print("\nTop 10 Products by Total Quantity Sold:")
print(top10_quantity)

print("\nTop 10 Products by Average Profit Margin:")
print(top10_profit_margin)

In [ ]:
latest_date = full_data['timestamp'].max()

rfm = full_data.groupby('customer_id').agg(
    recency=pd.NamedAgg(column='timestamp', aggfunc=lambda x: (latest_date - x.max()).days),
    frequency=('transaction_id', 'count'),
    monetary=('revenue', 'sum')
).reset_index()

bins = [0, 0.33, 0.66, 1]
labels = ['Low', 'Medium', 'High']

rfm['recency_bin'] = pd.qcut(rfm['recency'], q=bins, labels=labels, duplicates='drop').astype(str)
rfm['frequency_bin'] = pd.qcut(rfm['frequency'], q=bins, labels=labels, duplicates='drop').astype(str)
rfm['monetary_bin'] = pd.qcut(rfm['monetary'], q=bins, labels=labels, duplicates='drop').astype(str)

rfm['segment'] = rfm['recency_bin'] + '-' + rfm['frequency_bin'] + '-' + rfm['monetary_bin']

print("\nCustomer RFM Segmentation Sample")
print(rfm.head())

Part D: Advanced Operations

Efficient Operations Challenge


In [ ]:
def find_suspicious_transactions(full_data):
    """
    Identify suspicious transactions using vectorized operations.
    Returns: DataFrame with columns [transaction_id, customer_name, reason]
    """

    cond1 = (full_data['quantity'] > 100) & (full_data['price'] < 10)

    full_data['hour'] = full_data['timestamp'].dt.floor('H')
    purchase_counts = full_data.groupby(['customer_id', 'hour'])['transaction_id'].transform('count')
    cond2 = purchase_counts > 3


    suspicious_mask = cond1 | cond2

    full_data['reason'] = ''

    full_data.loc[cond1, 'reason'] = 'Qty > 100 and Price < 10'
    full_data.loc[cond2, 'reason'] = full_data.loc[cond2, 'reason'] + '; More than 3 purchases '

    result = full_data.loc[suspicious_mask, ['transaction_id', 'customer_name', 'reason']].copy()

    return result


Rolling Window Analysis


In [ ]:
import pandas as pd

# Normalize columns names
full_data.columns = full_data.columns.str.strip().str.lower()

# Aggregate daily revenue (timestamp to date)
full_data['date'] = full_data['timestamp'].dt.date
daily_revenue = full_data.groupby('date')['revenue'].sum().reset_index()

# Convert date to datetime type for rolling calculations
daily_revenue['date'] = pd.to_datetime(daily_revenue['date'])

# Calculate 7-day moving average of daily revenue with min_periods=1 to handle start edges
daily_revenue['7_day_revenue'] = daily_revenue['revenue'].rolling(window=7, min_periods=1).mean()

# Feature engineering for customers

# Convert signup_date to datetime
full_data['signup_date'] = pd.to_datetime(full_data['signup_date'])

# Get first transaction date per customer
first_transaction_date = full_data.groupby('customer_id')['timestamp'].min().reset_index()
first_transaction_date.rename(columns={'timestamp': 'first_transaction_date'}, inplace=True)

# Build initial customer features dataframe
customer_features = full_data[['customer_id', 'category', 'payment_method']].drop_duplicates(subset=['customer_id', 'category', 'payment_method'])
customer_features = customer_features.merge(first_transaction_date, on='customer_id')
customer_features = customer_features.merge(full_data[['customer_id', 'signup_date']].drop_duplicates(), on='customer_id')

# Calculate days_since_signup for each customer
customer_features['days_since_signup'] = (customer_features['first_transaction_date'] - customer_features['signup_date']).dt.days

# Calculate purchase_frequency: average days between purchases
purchase_freq = full_data.groupby('customer_id')['timestamp'].apply(lambda x: x.sort_values().diff().dt.days.mean()).reset_index()
purchase_freq.rename(columns={'timestamp': 'purchase_frequency'}, inplace=True)

customer_features = customer_features.merge(purchase_freq, on='customer_id')

# Category diversity: count unique categories per customer
category_diversity = full_data.groupby('customer_id')['category'].nunique().reset_index()
category_diversity.rename(columns={'category': 'category_diversity'}, inplace=True)
customer_features = customer_features.merge(category_diversity, on='customer_id')

# Preferred payment method
preferred_payment = full_data.groupby('customer_id')['payment_method'].agg(lambda x: x.mode().iat[0]).reset_index()
preferred_payment.rename(columns={'payment_method': 'preferred_payment'}, inplace=True)
customer_features = customer_features.merge(preferred_payment, on='customer_id')

# Display output samples
print(daily_revenue.head())
print(customer_features.head())


Part E: Code Quality & Efficiency

E1. Optimization


In [ ]:
import pandas as pd

def customer_lifetime_value(transactions_df, customer_id, discount_rate=0.1):
    """
    Calculate the lifetime value of a customer with monthly discounting.

    CLV = sum of (revenue * (1 - discount_rate) ^ month_from_first_purchase)

    Parameters:
    -----------
    transactions_df : pd.DataFrame
        Should contain 'customer_id', 'timestamp' (datetime), and 'revenue' columns.
    customer_id : int
        Customer ID you want to calculate CLV for.
    discount_rate : float
        Monthly discount rate (default 0.1)

    Returns:
    --------
    float
        The lifetime value calculated for the customer.
    """
    customer_data = transactions_df[transactions_df['customer_id'] == customer_id].copy()
    if customer_data.empty:
        return 0.0

    customer_data['timestamp'] = pd.to_datetime(customer_data['timestamp'])

    first_month = customer_data['timestamp'].min().to_period('M')

    customer_data['months_since_first'] = customer_data['timestamp'].dt.to_period('M').apply(lambda x: x.ordinal - first_month.ordinal)

    customer_data['discount_factor'] = (1 - discount_rate) ** customer_data['months_since_first']

    customer_data['discounted_revenue'] = customer_data['revenue'] * customer_data['discount_factor']

    clv = customer_data['discounted_revenue'].sum()

    return clv
